<a href="https://colab.research.google.com/github/TungAnhDep/Fundamental-Deep-Learning/blob/main/Convolutional_Neural_Network_LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchinfo

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torchinfo import summary

In [ ]:
num_classes = 10
lr = 0.001
num_epochs = 25


In [ ]:
"""
Lenet5:
5 layers: 2 convolutional, 3 fully-connected
- Input: gray scale image 32x32
- First convolutional:
  + 6 Filters size 5x5 -> output 28x28x6, followed by an activation function
  + Pooling -> 14x14x6
  + 16 filters size 5x5, and a pooling layer -> output feature map 5x5x16
- Second convolutional:
  + 120 filters size 5x5
-Flatten to 120 values
-Dense layer 120 ->84
-output dense layer 84 ->10
"""
class lenet5(nn.Module):
  def __init__(self, num_classes):
    super(lenet5, self).__init__()
    self.layer1 = nn.Sequential(
        nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(6),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
    )
    self.layer2 = nn.Sequential(
                nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0),
                nn.BatchNorm2d(16),
                nn.ReLU(),
                nn.MaxPool2d(kernel_size = 2, stride = 2)
                )
    self.fc = nn.Linear(400, 120)
    self.relu = nn.ReLU()
    self.fc1 = nn.Linear(120, 84)
    self.relu1 = nn.ReLU()
    self.fc2 = nn.Linear(84, num_classes)
  def forward(self,x):
    out = self.layer1(x) #batch_size,6, 14, 14
    out = self.layer2(out) #batch_size, 16 , 5, 5
    out = out.reshape(out.size(0), -1) #batch_size, 120

    out = self.fc(out) #batch_size, 120
    out = self.relu(out)
    out = self.fc1(out) #batch_size, 84
    out = self.relu1(out)
    out = self.fc2(out) #batch_size, num_classes
    return out


In [ ]:
train_dataset = torchvision.datasets.MNIST(root = '.',
                                               train = True,
                                               transform = transforms.Compose([
                                                      transforms.Resize((32,32)),
                                                      transforms.ToTensor(),
                                                      ]),
                                               download = True)
valid_dataset = torchvision.datasets.MNIST(root = '.',
                                          train = False,
                                          transform = transforms.Compose([
                                              transforms.Resize((32,32)),
                                              transforms.ToTensor(),

                                          ]),
                                          download = True)
train_loader =  DataLoader(dataset = train_dataset, shuffle = True, batch_size = 32)
train_n = len(train_loader.dataset)
valid_loader = DataLoader(dataset = valid_dataset, batch_size = 32)
valid_n = len(valid_loader.dataset)

100%|██████████| 9.91M/9.91M [00:00<00:00, 49.1MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.71MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 14.6MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 1.60MB/s]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
model = lenet5(num_classes).to(device)
model = torch.compile(model)
summary(model, (32,1,32,32))

Layer (type:depth-idx)                   Output Shape              Param #
OptimizedModule                          [32, 10]                  --
├─lenet5: 1-1                            [32, 10]                  --
│    └─Sequential: 2-1                   [32, 6, 14, 14]           --
│    │    └─Conv2d: 3-1                  [32, 6, 28, 28]           156
│    │    └─BatchNorm2d: 3-2             [32, 6, 28, 28]           12
│    │    └─ReLU: 3-3                    [32, 6, 28, 28]           --
│    │    └─MaxPool2d: 3-4               [32, 6, 14, 14]           --
│    └─Sequential: 2-2                   [32, 16, 5, 5]            --
│    │    └─Conv2d: 3-5                  [32, 16, 10, 10]          2,416
│    │    └─BatchNorm2d: 3-6             [32, 16, 10, 10]          32
│    │    └─ReLU: 3-7                    [32, 16, 10, 10]          --
│    │    └─MaxPool2d: 3-8               [32, 16, 5, 5]            --
│    └─Linear: 2-3                       [32, 120]                 48,120
│    └─

In [ ]:
loss_function = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters())


In [ ]:
def get_batch_accuracy(output, y, N):
    pred = output.argmax(dim=1, keepdim=True)
    correct = pred.eq(y.view_as(pred)).sum().item()
    return correct / N

In [ ]:
def validate():
  loss = 0
  accuracy = 0
  model.eval()
  with torch.no_grad():
    for x,y in valid_loader:
      output = model(x)
      loss+= loss_function(output, y).item()
      accuracy+=get_batch_accuracy(output, y, valid_n)
  print('FIXME - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
def train():
  loss = 0
  accuracy = 0
  model.train()
  for x,y in train_loader:
    output = model(x)
    optimizer.zero_grad()
    batch_loss = loss_function(output, y)
    batch_loss.backward()
    optimizer.step()
    loss+=batch_loss.item()
    accuracy+=get_batch_accuracy(output,y,train_n)
  print('FIXME - Loss: {:.4f} Accuracy: {:.4f}'.format(loss, accuracy))

In [ ]:
for epoch in range (num_epochs):
  print('Epoch: {}'.format(epoch))
  train()
  validate()

Epoch: 0


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


FIXME - Loss: 264.1369 Accuracy: 0.9584
FIXME - Loss: 16.5472 Accuracy: 0.9815
Epoch: 1
FIXME - Loss: 101.2174 Accuracy: 0.9828
FIXME - Loss: 13.6380 Accuracy: 0.9855
Epoch: 2
FIXME - Loss: 76.6353 Accuracy: 0.9871
FIXME - Loss: 13.3434 Accuracy: 0.9854
Epoch: 3
FIXME - Loss: 66.5393 Accuracy: 0.9888
FIXME - Loss: 12.7036 Accuracy: 0.9867
Epoch: 4
FIXME - Loss: 49.5454 Accuracy: 0.9915
FIXME - Loss: 10.4950 Accuracy: 0.9899
Epoch: 5
FIXME - Loss: 46.1027 Accuracy: 0.9922
FIXME - Loss: 11.3000 Accuracy: 0.9894
Epoch: 6
FIXME - Loss: 39.4881 Accuracy: 0.9935
FIXME - Loss: 8.9014 Accuracy: 0.9907
Epoch: 7
FIXME - Loss: 33.2497 Accuracy: 0.9939
FIXME - Loss: 10.5920 Accuracy: 0.9903
Epoch: 8
FIXME - Loss: 30.6454 Accuracy: 0.9949
FIXME - Loss: 9.1860 Accuracy: 0.9913
Epoch: 9
FIXME - Loss: 29.0298 Accuracy: 0.9952
FIXME - Loss: 10.5127 Accuracy: 0.9899
Epoch: 10
FIXME - Loss: 25.3604 Accuracy: 0.9955
FIXME - Loss: 10.7375 Accuracy: 0.9906
Epoch: 11
FIXME - Loss: 21.9715 Accuracy: 0.9960
FI